In [ ]:
import os
data_dir = El directorio donde guardo el data set
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')
f = open(fname)
data = f.read()
f.close()
lines = data.split(’\n’)
header = lines [0]. split(’,’)
lines = lines [1:]
print(header)
print(len(lines))